In [1]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown as md
import pandas as pd

InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

time_zone = 'Asia/Jakarta'

In [2]:
import data_source
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np

In [3]:
start_date = '2019-07-01'
end_date = '2023-06-30'

In [4]:
def prepare_tickers_per_sector(tickers_map: dict = None) -> pd.DataFrame:
    gathered_market_values: list[dict] = []
    if not tickers_map:
        tickers_map = data_source.tickers
    for sector in tickers_map.keys():
        gathered_market_values_per_sector: list[dict] = []
        for sector_member in tickers_map[sector]:
            gathered_market_values_per_sector.append({
                'sector': sector,
                'ticker': sector_member
            })
        dataframe_per_sector = pd.DataFrame(gathered_market_values_per_sector)
        dataframe_per_sector = dataframe_per_sector.sort_values(by='sector')
        gathered_market_values.extend(dataframe_per_sector.to_dict('records'))
    return pd.DataFrame(gathered_market_values)

In [5]:
tickers_per_sector = prepare_tickers_per_sector()
tickers_per_sector

,sector,ticker
0,energy,RIGS.JK
1,energy,DSSA.JK
2,energy,BIPI.JK
3,energy,HRUM.JK
4,energy,WINS.JK
5,energy,MBSS.JK
6,energy,BULL.JK
7,energy,PTIS.JK
8,energy,SMRU.JK
9,energy,ARII.JK


In [6]:
def generate_average_return_per_sector(data: pd.DataFrame, start_date: str, end_date: str) -> tuple[pd.DataFrame, dict]:
    start_time = datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=1)
    end_time = start_time + timedelta(days=2)
    if end_date:
        end_time = datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=1)

    updated_tickers = {}
    concat_object: dict[str, pd.Series] = {}
    for sector in data_source.tickers.keys():
        tickers_sectorized_df = data[data['sector'] == sector]
        if len(tickers_sectorized_df) < 1:
            continue
        tickers_sectorized_series = tickers_sectorized_df['ticker']
        tickers_sectorized = tickers_sectorized_series.to_list()
        prices_per_sector = yf.download(
            tickers=tickers_sectorized, start=start_time, end=end_time, interval='1wk')
        prices_per_sector_close = prices_per_sector['Close'].dropna(axis=1)
        updated_tickers[sector] = prices_per_sector_close.columns.tolist()
        return_of_tickers = prices_per_sector_close.pct_change()
        return_of_tickers = return_of_tickers.iloc[1:]
        sectoral_average_return = return_of_tickers.mean(axis=1)
        concat_object[sector] = sectoral_average_return
    average_return_per_sector = pd.concat(concat_object, axis=1)

    return average_return_per_sector, updated_tickers

In [7]:
average_return_per_sector, tickers_map_updated = generate_average_return_per_sector(tickers_per_sector, start_date, end_date)
tickers_per_sector = prepare_tickers_per_sector(tickers_map=tickers_map_updated)

[*********************100%***********************]  63 of 63 completed

1 Failed download:
- SUGI.JK: No timezone found, symbol may be delisted
[*********************100%***********************]  73 of 73 completed
[*********************100%***********************]  45 of 45 completed

1 Failed download:
- TRIL.JK: No timezone found, symbol may be delisted
[*********************100%***********************]  71 of 71 completed
[*********************100%***********************]  99 of 99 completed

3 Failed downloads:
- TRIO.JK: No timezone found, symbol may be delisted
- NIPS.JK: No timezone found, symbol may be delisted
- HDTX.JK: No timezone found, symbol may be delisted
[*********************100%***********************]  17 of 17 completed
[*********************100%***********************]  96 of 96 completed
[*********************100%***********************]  59 of 59 completed
[*********************100%***********************]  13 of 13 completed
[*********************100%*********

In [8]:
average_return_per_sector

,energy,basic-materials,industrials,consumer-non-cyclicals,consumer-cyclicals,healthcare,financials,properties-real-estate,technology,infrastructures,transportation-logistics
Date,,,,,,,,,,,
2019-07-08,-0.026025,-0.010457,-0.003023,-0.018503,-0.000034,-0.003912,0.002541,0.010148,0.003991,-0.010750,0.018217
2019-07-15,-0.011055,0.012831,-0.000225,-0.000747,0.005351,-0.002187,-0.000709,0.000561,0.002834,0.002107,-0.005864
2019-07-22,-0.020184,0.022359,0.018839,0.006168,-0.012992,0.006322,-0.013785,-0.005920,-0.035387,-0.002189,-0.004362
2019-07-29,-0.000140,-0.002175,-0.010433,-0.002071,-0.001377,-0.002486,0.013112,0.000330,-0.037220,-0.017900,0.004746
2019-08-05,-0.017613,0.008301,-0.019499,-0.001102,-0.002330,-0.001501,-0.010738,-0.008558,-0.015115,-0.002755,0.013142
2019-08-12,-0.019534,-0.003277,0.007592,0.008233,-0.004476,0.007882,0.016605,0.013504,-0.002459,-0.004434,0.000248
2019-08-19,-0.009432,-0.008003,-0.008454,-0.010812,-0.005742,-0.009093,0.003600,0.001038,-0.012705,0.004453,-0.009565
2019-08-26,0.013532,0.020612,0.005373,0.000196,0.005141,0.002203,-0.005979,-0.003562,-0.042104,-0.004523,0.006055
2019-09-02,0.012928,0.007353,-0.003456,-0.003648,0.003250,-0.023693,-0.003646,0.002274,-0.030189,-0.020183,-0.006819


In [9]:
tickers_per_sector # this is updated

,sector,ticker
0,energy,ADRO.JK
1,energy,KKGI.JK
2,energy,KOPI.JK
3,energy,LEAD.JK
4,energy,MBAP.JK
5,energy,MBSS.JK
6,energy,MEDC.JK
7,energy,MITI.JK
8,energy,MTFN.JK
9,energy,MYOH.JK


In [10]:
correlations = average_return_per_sector.corr()
correlations

,energy,basic-materials,industrials,consumer-non-cyclicals,consumer-cyclicals,healthcare,financials,properties-real-estate,technology,infrastructures,transportation-logistics
energy,1.000000,0.697691,0.623841,0.721989,0.214611,0.428663,0.289064,0.550521,0.304221,0.622315,0.435192
basic-materials,0.697691,1.000000,0.649994,0.764378,0.266023,0.545658,0.265251,0.661004,0.391744,0.803033,0.513817
industrials,0.623841,0.649994,1.000000,0.638643,0.164028,0.389585,0.261671,0.579170,0.411635,0.635707,0.486553
consumer-non-cyclicals,0.721989,0.764378,0.638643,1.000000,0.212739,0.496525,0.272081,0.612613,0.343010,0.730455,0.451693
consumer-cyclicals,0.214611,0.266023,0.164028,0.212739,1.000000,0.160752,0.049206,0.107214,0.036730,0.304024,0.154631
healthcare,0.428663,0.545658,0.389585,0.496525,0.160752,1.000000,0.141612,0.357615,0.269940,0.416815,0.222145
financials,0.289064,0.265251,0.261671,0.272081,0.049206,0.141612,1.000000,0.217006,0.136466,0.278224,0.160243
properties-real-estate,0.550521,0.661004,0.579170,0.612613,0.107214,0.357615,0.217006,1.000000,0.411943,0.681025,0.498740
technology,0.304221,0.391744,0.411635,0.343010,0.036730,0.269940,0.136466,0.411943,1.000000,0.429360,0.291346
infrastructures,0.622315,0.803033,0.635707,0.730455,0.304024,0.416815,0.278224,0.681025,0.429360,1.000000,0.542273


In [11]:
desired_sector = 'basic-materials'
correlation_for_desired_sector = correlations[desired_sector].to_frame()

correlation_column_name = "correlation"
first_column_name = correlation_for_desired_sector.columns[0]
correlation_for_desired_sector.rename(columns={first_column_name: correlation_column_name}, inplace=True)
correlation_for_desired_sector = correlation_for_desired_sector.sort_values(by=correlation_column_name, ascending=False)
correlation_for_desired_sector

,correlation
basic-materials,1.000000
infrastructures,0.803033
consumer-non-cyclicals,0.764378
energy,0.697691
properties-real-estate,0.661004
industrials,0.649994
healthcare,0.545658
transportation-logistics,0.513817
technology,0.391744
consumer-cyclicals,0.266023


In [12]:
def set_relativity_label(data: pd.DataFrame) -> pd.DataFrame:
    first_column_name = data.columns[0]
    data = data.sort_values(first_column_name, ascending=False)

    length_of_data = len(data)
    half_point = length_of_data / 2

    data.insert(loc=len(data.columns), column='is_more_related', value=np.nan)
    for i in range(len(data)):
        is_more_related = i < half_point
        data.loc[data.index[i], 'is_more_related'] = is_more_related

    return data

In [13]:
relativity_map = set_relativity_label(data=correlation_for_desired_sector)
relativity_map

,correlation,is_more_related
basic-materials,1.000000,True
infrastructures,0.803033,True
consumer-non-cyclicals,0.764378,True
energy,0.697691,True
properties-real-estate,0.661004,True
industrials,0.649994,True
healthcare,0.545658,False
transportation-logistics,0.513817,False
technology,0.391744,False
consumer-cyclicals,0.266023,False


In [14]:
def set_tickers_relativity_label(data: pd.DataFrame, relativity_map: pd.DataFrame) -> pd.DataFrame:
    data.insert(loc=len(data.columns), column='is_more_related', value=np.nan)
    for i in range(len(data)):
        sector = data.loc[data.index[i], 'sector']
        is_more_related = relativity_map.loc[sector]['is_more_related']
        data.loc[data.index[i], 'is_more_related'] = is_more_related

    return data

In [15]:
tickers = set_tickers_relativity_label(data=tickers_per_sector, relativity_map=relativity_map)
tickers

,sector,ticker,is_more_related
0,energy,ADRO.JK,True
1,energy,KKGI.JK,True
2,energy,KOPI.JK,True
3,energy,LEAD.JK,True
4,energy,MBAP.JK,True
5,energy,MBSS.JK,True
6,energy,MEDC.JK,True
7,energy,MITI.JK,True
8,energy,MTFN.JK,True
9,energy,MYOH.JK,True


In [16]:
md(f'number of more related tickers: {len(tickers[tickers["is_more_related"] == True])}')
md(f'number of less related tickers: {len(tickers[tickers["is_more_related"] == False])}')

number of more related tickers: 357

number of less related tickers: 243

In [18]:
with pd.ExcelWriter('output_preprocessing.xlsx') as excel_writer:
    tickers.to_excel(excel_writer=excel_writer, sheet_name='tickers')
    relativity_map.to_excel(excel_writer=excel_writer, sheet_name='relativities')